### Classificação binária brest cancer - classificar somente um registro e salvar o classificador

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
torch.__version__

'2.0.0+cu118'

In [2]:
np.random.seed(123)
torch.manual_seed(123)

In [3]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [4]:
previsores = torch.tensor(np.array(previsores), dtype = torch.float)
classe = torch.tensor(np.array(classe), dtype = torch.float)

In [5]:
type(previsores)

torch.Tensor

In [8]:
# Transformando os dados para Tensores
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classe), batch_size=10, shuffle = True)

### Contrução do Modelo

In [9]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__() 
        self.dense0 = nn.Linear(30, 8)
        # Mesmos parâmetors usados pelo kernel initializer do Keras [https://keras.io/initializers/, ver sessão RandomNormal]
        torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std= 0.05)
        self.dense1 = nn.Linear(8, 8)
        torch.nn.init.normal_(self.dense1.weight, mean = 0.0, std= 0.05)
        self.dense2 = nn.Linear(8, 1)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.output = nn.Sigmoid()
        
    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense2(X)
        X = self.output(X)
        return X

In [10]:
classificador = classificador_torch()

In [11]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(classificador.parameters(), lr = 0.001, 
                             weight_decay = 0.0001)

### Treinando Modelo

In [12]:
for epoch in range(100):
    running_loss = 0.
    
    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()        

        outputs = classificador(inputs)
        loss = criterion(outputs, labels)
        loss.backward()    
        optimizer.step()

        running_loss += loss.item()

    print('Época %3d: perda %.5f' % (epoch+1, running_loss/len(train_loader)))

Época   1: perda 1.65405
Época   2: perda 0.83266
Época   3: perda 0.68309
Época   4: perda 0.61071
Época   5: perda 0.58727
Época   6: perda 0.59212
Época   7: perda 0.54197
Época   8: perda 0.52175
Época   9: perda 0.51248
Época  10: perda 0.51385
Época  11: perda 0.50309
Época  12: perda 0.47485
Época  13: perda 0.47657
Época  14: perda 0.45629
Época  15: perda 0.44756
Época  16: perda 0.44154
Época  17: perda 0.41747
Época  18: perda 0.39531
Época  19: perda 0.42316
Época  20: perda 0.37898
Época  21: perda 0.40490
Época  22: perda 0.38421
Época  23: perda 0.38859
Época  24: perda 0.39575
Época  25: perda 0.37081
Época  26: perda 0.37289
Época  27: perda 0.36106
Época  28: perda 0.37193
Época  29: perda 0.37447
Época  30: perda 0.35007
Época  31: perda 0.36657
Época  32: perda 0.31675
Época  33: perda 0.32431
Época  34: perda 0.33630
Época  35: perda 0.36814
Época  36: perda 0.35728
Época  37: perda 0.33705
Época  38: perda 0.31823
Época  39: perda 0.32361
Época  40: perda 0.33200


### Classificar somente um registro

In [13]:
novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [14]:
classificador.eval()

classificador_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [15]:
previsao = classificador(novo)

In [16]:
previsao

tensor([[1.]], grad_fn=<SigmoidBackward0>)

In [17]:
previsao = previsao.detach()

In [18]:
previsao

tensor([[1.]])

In [19]:
previsao = previsao.numpy()

In [20]:
type(previsao)

numpy.ndarray

In [21]:
previsao

array([[1.]], dtype=float32)

In [22]:
previsao = (previsao > 0.5)
previsao

array([[ True]])

In [23]:
# Salvando o classificador
# Quando salvar, você precisa chamar classificador.state_dict() (com os parêntese no final), 
# ao invés de classificador.state_dict
classificador.state_dict()

OrderedDict([('dense0.weight',
              tensor([[-6.3329e-01, -6.4088e-03, -3.8681e-01, -1.0687e-01,  9.4179e-02,
                       -2.8071e-02,  5.8217e-02, -5.5828e-02,  1.4919e-02,  5.5338e-02,
                        8.5171e-03, -1.0727e-03,  3.1089e-03, -1.7085e-02,  3.1640e-02,
                        2.3500e-01, -3.9340e-01, -1.7852e-01, -6.7033e-02,  5.4838e-01,
                       -4.7472e-01,  2.4388e-02, -3.2421e-01,  1.8188e-01,  1.4345e-02,
                        3.0411e-02,  1.2222e-02,  4.2423e-02,  1.6629e-02,  6.2451e-01],
                      [ 9.6147e-02,  8.9823e-03,  6.2657e-02, -2.7833e-02,  2.5520e-02,
                       -1.4683e-02,  8.4852e-09, -5.5642e-02, -4.3206e-02,  2.9459e-04,
                        3.3087e-02,  1.5457e-03, -3.6564e-02,  7.3219e-02, -9.4868e-11,
                       -5.2539e-07,  3.7381e-03,  2.0600e-09,  6.6002e-09,  2.5737e-10,
                       -7.0341e-02,  3.3297e-02, -3.6295e-02, -3.3981e-02, -7.6689e-02,


In [24]:
# Para salvar o classificador com a versão 1.5.0 do PyTorch, use o código abaixo

torch.save(classificador.state_dict(), 'checkpoint.pth')

In [25]:
state_dict = torch.load('/content/checkpoint.pth')

In [26]:
#estrutura da rede neural
state_dict

OrderedDict([('dense0.weight',
              tensor([[-6.3329e-01, -6.4088e-03, -3.8681e-01, -1.0687e-01,  9.4179e-02,
                       -2.8071e-02,  5.8217e-02, -5.5828e-02,  1.4919e-02,  5.5338e-02,
                        8.5171e-03, -1.0727e-03,  3.1089e-03, -1.7085e-02,  3.1640e-02,
                        2.3500e-01, -3.9340e-01, -1.7852e-01, -6.7033e-02,  5.4838e-01,
                       -4.7472e-01,  2.4388e-02, -3.2421e-01,  1.8188e-01,  1.4345e-02,
                        3.0411e-02,  1.2222e-02,  4.2423e-02,  1.6629e-02,  6.2451e-01],
                      [ 9.6147e-02,  8.9823e-03,  6.2657e-02, -2.7833e-02,  2.5520e-02,
                       -1.4683e-02,  8.4852e-09, -5.5642e-02, -4.3206e-02,  2.9459e-04,
                        3.3087e-02,  1.5457e-03, -3.6564e-02,  7.3219e-02, -9.4868e-11,
                       -5.2539e-07,  3.7381e-03,  2.0600e-09,  6.6002e-09,  2.5737e-10,
                       -7.0341e-02,  3.3297e-02, -3.6295e-02, -3.3981e-02, -7.6689e-02,


In [27]:
classificador.load_state_dict(state_dict)

<All keys matched successfully>

In [28]:
novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [29]:
classificador.eval()
previsao = classificador(novo)
previsao = (previsao.detach().numpy() > 0.5)
previsao

array([[ True]])

In [30]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [31]:
previsores = torch.tensor(np.array(previsores), dtype = torch.float)
classe = torch.tensor(np.array(classe), dtype = torch.float)

In [32]:
previsoes = classificador.forward(previsores).detach()

In [33]:
previsoes

tensor([[5.2292e-07],
        [9.2073e-05],
        [1.0848e-03],
        [1.5193e-04],
        [1.2827e-02],
        [5.7314e-01],
        [7.7519e-05],
        [2.3465e-01],
        [3.0953e-01],
        [6.9902e-03],
        [1.2647e-01],
        [4.5154e-03],
        [2.5287e-01],
        [6.8647e-01],
        [6.8647e-01],
        [4.6997e-01],
        [3.0779e-02],
        [3.7884e-03],
        [3.9827e-08],
        [9.6103e-01],
        [6.8647e-01],
        [1.0000e+00],
        [3.9961e-01],
        [2.4730e-08],
        [1.7898e-08],
        [2.7582e-04],
        [4.2254e-01],
        [1.6693e-02],
        [1.7407e-03],
        [1.9918e-01],
        [1.4824e-03],
        [5.1415e-02],
        [1.2670e-02],
        [7.5736e-05],
        [5.3576e-03],
        [9.8634e-02],
        [6.8647e-01],
        [9.9928e-01],
        [6.8647e-01],
        [6.3165e-01],
        [6.6119e-01],
        [8.2413e-01],
        [1.3902e-04],
        [1.4843e-01],
        [6.8647e-01],
        [1

In [35]:
import torch.nn.functional as F

In [36]:
F.binary_cross_entropy(previsoes, classe).numpy()

array(0.19902807, dtype=float32)

In [37]:
accuracy_score(classe.numpy(), (previsoes > 0.5).numpy())

0.9332161687170475